In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
# from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
# 출력 예쁘게 하기
from rich.console import Console
from rich.table import Table

console = Console()

def rich_docs(docs, title="Retriever Results", max_len=140):
    table = Table(title=title, width = 500)
    table.add_column("#", justify="right")
    table.add_column("Source")
    table.add_column("Page", justify="right")
    table.add_column("Preview")

    for i, d in enumerate(docs, 1):
        m = d.metadata or {}
        src = (m.get("source","") or "").split("/")[-1]
        page = str(m.get("page_label", m.get("page",0)+1))
        text = (d.page_content or "").strip().replace("\n", " ")
        content = (text[:max_len] + ("…" if len(text) > max_len else ""))
        table.add_row(str(i), src, page, content)

    console.print(table)

# 1. 문서 저장

## 1. 문서 로드

In [15]:
from langchain_community.document_loaders import PyMuPDFLoader, PyPDFLoader, PDFPlumberLoader, UnstructuredPDFLoader

In [14]:
file_path = '../data/CN7N_2026_ko_KR.pdf'

### PyPDFLoader

In [ ]:
print("PyPDFLoader")
loader = PyPDFLoader(file_path)
docs = loader.load()

In [ ]:
print(docs[1].page_content)
# 이미지 안에 있는 글자와 일반 글자가 혼재되어있음

내용 찾기 방법 설명
에어컨
사용방법이
어디지...
이 스위치가
뭐지?
점검방법이
어느 장에 있지
목차를 활용하세요.
그림 목차 (내관도 I ) 를 활용하세요.
색인(명칭) 목차 활용하세요.
내용으로 찾을 때
명칭을 모를 때
명칭으로 찾을 때


In [12]:
print(docs[39].page_content)
# 왼쪽 > 오른쪽으로 읽어야하는데
# 오른쪽 > 왼쪽 순으로 읽는 경우가 많음

3-12
안전 장치
 • 수동 온도 조절
  좌석의 스위치를 조작하면 아래와 같이 온도 조절
이 됩니다
- 앞좌석
- 뒷좌석
 • 자동 온도 조절
 저온 화상 방지를 위해 좌석 히터를 수동으로 켠 다음
부터는 자동으로 좌석 온도가 조절됩니다.
- 앞좌석
- 뒷좌석
 다시 고온으로 사용하려면, 스위치를 조작하여 조
절하십시오. 하지만, 다시 자동으로 좌석 온도가 조
절됩니다. 
 • 좌석 히터 작동 중 버튼을 1.5초 이상 누르면 좌석 
히터를 끌 수 있습니다. (사양적용시)
 • 시동을 껐다가 다시 시동을 걸면 좌석 히터는 항상 
OFF 상태가 됩니다.  
 단, 운전석 히터는 자동 쾌적 제어 기능이 켜져 있으
면 내/외기 온도 및 공조 설정 온도에 따라 ON/
OFF 됩니다.
 • 운전석 자동 쾌적 제어 (사양적용시)
 인포테인먼트 시스템 환경 설정 중 스마트 열선/통
풍 기능을 설정하면 차량의 내/외기 온도 및 공조 
설정 온도에 따라 자동으로 제어됩니다.
 업데이트로 인해 인포테인먼트 시스템의 상세 설정
이 변경될 수 있습니다. 자세한 설정 방법은 인포테
인먼트 웹 매뉴얼을 참고하십시오.
 ̰ 좌석 히터 (열선)
 • 시동이 걸린 상태에서 해당 좌석 히터 스위치를 작
동하면 강약 조절 표시등이 켜져 사용 중임을 나타
내고 해당 좌석이 따뜻해집니다.
 • 좌석 히터 작동(표시등 켜짐) 상태에서 좌석 온도가 
일정 온도 이상 높아지면 자동으로 작동을 멈추고, 
온도가 내려가면 다시 작동하게 됩니다.
OCN7N033145OCN7N033145
  ̰̰ 앞좌석앞좌석
  •• A타입A타입
OCN7N033146OCN7N033146
  ̰̰ 앞좌석앞좌석
  •• B타입B타입
OFF → 약( ) → 중( ) → 강( )
↑
OFF → 약( ) → 강( )
↑
OCN7030025OCN7030025
  ̰̰ 뒷좌석(사양 적용 시)뒷좌석(사양 적용 시)
OFF → 약( ) → 강( )
30분 후
↑
OFF → 약( ) → 중( ) → 강( ) 
60분 후 30분 후
↑↑
↑


### DedocPDFLoader

In [ ]:
# python 3.14 이상 설치
from langchain_community.document_loaders import DedocPDFLoader

loader = DedocPDFLoader(
    file_path,
    need_header_footer_analysis=True,
    is_one_column_document="auto",
    pdf_with_text_layer="auto_tabby",
    language="kor+eng",
    split="page",
)
docs = loader.load()

### PyMuPDFLoader

In [ ]:
print("문서 로드중")
loader = PyMuPDFLoader(file_path, extract_tables="markdown")
docs = loader.load()

문서 로드중


In [25]:
print(docs[1].page_content)
# 이미지 안에 있는 글자와 일반 글자가 혼재되어있음

내용 찾기 방법 설명
에어컨
사용방법이
어디지...
이 스위치가
뭐지?
점검방법이
어느 장에 있지
목차를 활용하세요.
그림 목차 (내관도 I ) 를 활용하세요.
색인(명칭) 목차 활용하세요.
내용으로 찾을 때
명칭을 모를 때
명칭으로 찾을 때


In [26]:
print(docs[39].page_content)
# 왼쪽 > 오른쪽으로 읽어야하는데
# 오른쪽 > 왼쪽 순으로 읽는 경우가 많음

3-12
안전 장치
	
• 수동 온도 조절
 	
좌석의 스위치를 조작하면 아래와 같이 온도 조절
이 됩니다
-	 앞좌석
-	 뒷좌석
	
• 자동 온도 조절
	
저온 화상 방지를 위해 좌석 히터를 수동으로 켠 다음
부터는 자동으로 좌석 온도가 조절됩니다.
-	 앞좌석
-	 뒷좌석
	
다시 고온으로 사용하려면, 스위치를 조작하여 조
절하십시오. 하지만, 다시 자동으로 좌석 온도가 조
절됩니다. 
	
• 좌석 히터 작동 중 버튼을 1.5초 이상 누르면 좌석 
히터를 끌 수 있습니다. (사양적용시)
	
• 시동을 껐다가 다시 시동을 걸면 좌석 히터는 항상 
OFF 상태가 됩니다.  
	
단, 운전석 히터는 자동 쾌적 제어 기능이 켜져 있으
면 내/외기 온도 및 공조 설정 온도에 따라 ON/
OFF 됩니다.
	
• 운전석 자동 쾌적 제어 (사양적용시)
	
인포테인먼트 시스템 환경 설정 중 스마트 열선/통
풍 기능을 설정하면 차량의 내/외기 온도 및 공조 
설정 온도에 따라 자동으로 제어됩니다.
	
업데이트로 인해 인포테인먼트 시스템의 상세 설정
이 변경될 수 있습니다. 자세한 설정 방법은 인포테
인먼트 웹 매뉴얼을 참고하십시오.
	
̰ 좌석 히터 (열선)
	
• 시동이 걸린 상태에서 해당 좌석 히터 스위치를 작
동하면 강약 조절 표시등이 켜져 사용 중임을 나타
내고 해당 좌석이 따뜻해집니다.
	
• 좌석 히터 작동(표시등 켜짐) 상태에서 좌석 온도가 
일정 온도 이상 높아지면 자동으로 작동을 멈추고, 
온도가 내려가면 다시 작동하게 됩니다.
OCN7N033145
OCN7N033145
	̰
	 앞좌석
앞좌석
	•
	 A타입
A타입
OCN7N033146
OCN7N033146
	̰
	 앞좌석
앞좌석
	•
	 B타입
B타입
OFF → 약(
) → 중(
) → 강(
)
↑
OFF → 약(
) → 강(
)
↑
OCN7030025
OCN7030025
	̰
	 뒷좌석(사양 적용 시)
뒷좌석(사양 적용 시)
OFF → 약(
) → 강(
)
30분 후
↑
OFF → 약(
) → 중(
)

### pymupdf4llm

In [32]:
import pymupdf4llm

loader = pymupdf4llm.LlamaMarkdownReader()
docs = loader.load_data(file_path)

c:\Potenup\LLM-Study\.venv\Lib\site-packages\pydantic\_internal\_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


Successfully imported LlamaIndex


In [36]:
len(docs)

436

In [34]:
print(docs[1])

Doc ID: 50f59f5b-37f0-4894-9de9-4a2764f50590
Text: ###### 내용 찾기 방법 설명  내용으로 찾을 때   명칭을 모를 때   명칭으로 찾을 때    목차를
활용하세요.   그림 목차 (내관도 I ) 를 활용하세요.   색인(명칭) 목차 활용하세요.


In [35]:
print(docs[39])

Doc ID: 416bfa72-b5fd-4fd0-9b37-b1cfe46f9dd5
Text: 안전 장치   ̰ 좌석 히터 (열선)       - 시동이 걸린 상태에서 해당 좌석 히터 스위치를 작  동하면 강약
조절 표시등이 켜져 사용 중임을 나타  내고 해당 좌석이 따뜻해집니다.  - 좌석 히터 작동(표시등 켜짐) 상태에서 좌석
온도가  일정 온도 이상 높아지면 자동으로 작동을 멈추고, 온도가 내려가면 다시 작동하게 됩니다.   3-12     - 수동
온도 조절  좌석의 스위치를 조작하면 아래와 같이 온도 조절  이 됩니다   - 앞좌석  OFF → 약 ~~(~~ ~~)~~
→ ~~중(~~ ~~)~~ → 강 ~~(~~ ~~)~~ ~~↑~~    - 뒷좌석   OFF → 약 ~~(~~ ~~)~~ →
~~강...


In [ ]:
import pymupdf4llm

md = pymupdf4llm.to_markdown(file_path)  # 멀티컬럼 보존 MD
# 여기서 헤더/푸터 패턴 제거 → 문단 단위 chunk → 임베딩

In [30]:
print(md)

###### 안전 및 차량 손상 경고

본 취급설명서는 고객 및 차량의 안전과 관련한 심각한 위험과 제품 사용에 대한 올바른 정보를 사
전에 알리는 안전경고 표시입니다. 지시사항은 반드시 숙지하여 지켜주십시오.



경고, 주의표시


선택 또는 미장착 사양표시



경고, 주의가 있는 문장 및 진하게 표시되어 있는 부분은
특히 유념하십시오.


주 의


차량이 고장나거나 손상될 우려가 있는 경우의 주의 표

시입니다.


경 고


사람이 다치거나 사망의 우려가 있는 경우의 경고 표시

입니다.


_**i**_ [알아두기]


차량 용어 또는 추가 설명이 필요한 정보 표시입니다.


안전을 위해 반드시 지켜야 하는 금지 표시입니다.


본 취급설명서에는 모든 트림모델 및 선택사양을 포함

하여 설명하고 있습니다.
따라서 고객님 차량에 장착되지 않은 사양이 설명될 수
있습니다.


###### 내용 찾기 방법 설명

내용으로 찾을 때


명칭을 모를 때


명칭으로 찾을 때



목차를 활용하세요.


그림 목차 (내관도 I ) 를 활용하세요.


색인(명칭) 목차 활용하세요.


제작결함 안내


제작결함 안내
(제50조 관련)



자동차제작자등(부품제작자등) : 현대자동차(주)
주 소 : 서울특별시 서초구 헌릉로 12(양재동)

연락처 : 080-600-6000


귀하의 자동차 또는 자동차부품에 잦은 고장 등의 문제
로 교통사고를 유발할 수 있는 결함이 있다고 판단되면,
자기 및 다른 사람의 안전을 위하여 즉시 현대자동차(주)
와 제작결함조사를 시행하는 한국교통안전공단 자동차
안전연구원에 연락하여 주시기 바랍니다.


한국교통안전공단 자동차안전연구원은 소비자 불만사
항 등을 접수하여 분석한 후 해당 자동차 또는 자동차부
품에 제작결함의 가능성이 있다고 판단되는 경우 제작
결함조사를 실시하여 해당 제작사에게 제작결함시정
(recall) 등의 조치를 취할 것입니다.


한국교통안전공단 자동차안전연구원의 자동차 또는 자
동차부품 결함 등 소비자 불만 접수

### PDFPlumberLoader

In [39]:
from langchain_community.document_loaders import PDFPlumberLoader

In [41]:
loader = PDFPlumberLoader(file_path)
docs = loader.load()

In [43]:
print(docs[39].page_content)

안전 장치
̰ 좌석 히터 (열선) • 수동 온도 조절
좌석의 스위치를 조작하면 아래와 같이 온도 조절
̰̰ 앞앞좌좌석석
이 됩니다
•• AA타타입입
- 앞좌석
OFF → 약( ) → 중( ) → 강( )
↑
- 뒷좌석
OFF → 약( ) → 강( )
↑
OOCCNN77NN003333114455 • 자동 온도 조절
저온 화상 방지를 위해 좌석 히터를 수동으로 켠 다음
̰̰ 앞앞좌좌석석
•• BB타타입입 부터는 자동으로 좌석 온도가 조절됩니다.
- 앞좌석
OFF → 약( ) → 중( ) → 강( )
↑ ↑
60분 후 30분 후
- 뒷좌석
OFF → 약( ) → 강( )
↑
OOCCNN77NN003333114466
30분 후
̰̰ 뒷뒷좌좌석석((사사양양 적적용용 시시)) 다시 고온으로 사용하려면, 스위치를 조작하여 조
절하십시오. 하지만, 다시 자동으로 좌석 온도가 조
절됩니다.
• 좌석 히터 작동 중 버튼을 1.5초 이상 누르면 좌석
히터를 끌 수 있습니다. (사양적용시)
• 시동을 껐다가 다시 시동을 걸면 좌석 히터는 항상
OFF 상태가 됩니다.
단, 운전석 히터는 자동 쾌적 제어 기능이 켜져 있으
면 내/외기 온도 및 공조 설정 온도에 따라 ON/
OOCCNN77003300002255
OFF 됩니다.
• 시동이 걸린 상태에서 해당 좌석 히터 스위치를 작 • 운전석 자동 쾌적 제어 (사양적용시)
동하면 강약 조절 표시등이 켜져 사용 중임을 나타
인포테인먼트 시스템 환경 설정 중 스마트 열선/통
내고 해당 좌석이 따뜻해집니다.
풍 기능을 설정하면 차량의 내/외기 온도 및 공조
• 좌석 히터 작동(표시등 켜짐) 상태에서 좌석 온도가 설정 온도에 따라 자동으로 제어됩니다.
일정 온도 이상 높아지면 자동으로 작동을 멈추고,
업데이트로 인해 인포테인먼트 시스템의 상세 설정
온도가 내려가면 다시 작동하게 됩니다.
이 변경될 수 있습니다. 자세한 설정 방법은 인포테
인먼트 웹 매뉴얼을 참고하십시오.
3-12
↑



### custom

In [54]:
# put this at the top of your script/module
from pathlib import Path
import sys

def add_root_for_utils(max_up=8, utils_dir_name="utils"):
    try:
        start = Path(__file__).resolve().parent   # 일반 .py 파일
    except NameError:
        start = Path.cwd().resolve()              # Jupyter/REPL 등

    p = start
    for _ in range(max_up + 1):
        if (p / utils_dir_name).is_dir():
            root = str(p)
            if root not in sys.path:
                sys.path.insert(0, root)
            return p
        if p.parent == p:
            break
        p = p.parent

    raise RuntimeError(f"상위 {max_up}단계 내에서 '{utils_dir_name}' 폴더를 찾지 못했어요.")

# 실행
add_root_for_utils(max_up=8)

WindowsPath('C:/Potenup/LLM-Study')

In [57]:
from utils import custon_pdf_loader

file_path = '../data/CN7N_2026_ko_KR.pdf'
reulst, info = custon_pdf_loader.extract_pdf_with_cluster_order(file_path, detect_repeating=False)

In [58]:
print(len(reulst))
print(len(info))

436
436


In [52]:
print(docs[39])

안전 장치
̰ 좌석 히터 (열선) ̰ 앞좌석 앞좌석 • A타입 A타입
OCN7030025 OCN7030025 • 시동이 걸린 상태에서 해당 좌석 히터 스위치를 작 동하면 강약 조절 표시등이 켜져 사용 중임을 나타 내고 해당 좌석이 따뜻해집니다. • 좌석 히터 작동(표시등 켜짐) 상태에서 좌석 온도가 일정 온도 이상 높아지면 자동으로 작동을 멈추고, 온도가 내려가면 다시 작동하게 됩니다.
3-12
̰ 앞좌석 앞좌석 • B타입 B타입
̰ 뒷좌석(사양 적용 시) 뒷좌석(사양 적용 시)
OCN7N033145 OCN7N033145
OCN7N033146 OCN7N033146
• 수동 온도 조절 좌석의 스위치를 조작하면 아래와 같이 온도 조절 이 됩니다 -	 앞좌석 OFF → 약( ) → 중( ) → 강( ↑ -	 뒷좌석 OFF → 약( ) → 강( ) ↑ • 자동 온도 조절 저온 화상 방지를 위해 좌석 히터를 수동으로 켠 다음 부터는 자동으로 좌석 온도가 조절됩니다. -	 앞좌석 ↑ OFF → 약( ) → 중( ) → 강( ↑ ↑ 60분 후 30분 후 -	 뒷좌석 OFF → 약( ) → 강( ) ↑ 30분 후 다시 고온으로 사용하려면, 스위치를 조작하여 조 절하십시오. 하지만, 다시 자동으로 좌석 온도가 조 절됩니다. • 좌석 히터 작동 중 버튼을 1.5초 이상 누르면 좌석 히터를 끌 수 있습니다. (사양적용시) • 시동을 껐다가 다시 시동을 걸면 좌석 히터는 항상 OFF 상태가 됩니다. 단, 운전석 히터는 자동 쾌적 제어 기능이 켜져 있으 면 내/외기 온도 및 공조 설정 온도에 따라 ON/ OFF 됩니다. • 운전석 자동 쾌적 제어 (사양적용시) 인포테인먼트 시스템 환경 설정 중 스마트 열선/통 풍 기능을 설정하면 차량의 내/외기 온도 및 공조 설정 온도에 따라 자동으로 제어됩니다. 업데이트로 인해 인포테인먼트 시스템의 상세 설정 이 변경될 수 있습니다. 자세한 설정 방법은 인포테 인먼트 웹 매뉴얼을 참고하십시오.
)
)


In [53]:
print(docs[40])

경 고 히터 사용 중 아래의 사람은 열이 나거나 저온화상을 입 을 수 있으니 주의하십시오. 1.	 유아, 어린이, 노인, 신체 부자유자 또는 기타 질병 이 있는 분 2.	 피부가 약한 분 3.	 과로한 분 4.	 과음한 분 5.	 졸음이 올 수 있는 약(수면제, 감기약)을 복용한 분 i 알아두기 저온화상이란? 피부가 40~50℃ 정도의 비교적 낮은 온도에 장시 간 노출되었을 때 열에 의하여 화상을 입는 것을 말 하며 통증이나 열을 그다지 느끼지 못하는 사이에 일어나므로 주의해야 합니다. 경 고 좌석 커버를 임의로 교체하지 마십시오. 히터가 손상되 어 화재가 발생할 위험이 있습니다. 주 의 • 무거운 물건이나 날카로운 물건을 좌석에 올려놓지 마십시오. • 좌석을 세척할 때 시너, 벤젠, 알코올, 가솔린 등의 유기용제를 사용하지 마십시오. 좌석 히터(열선) 시 스템이 손상될 수 있습니다. • 과열 방지를 위해, 좌석에 모포나 방석 등을 올려놓 지 마십시오. • 물 등의 액체를 좌석에 쏟았을 경우, 좌석 히터(열 선)를 끄고 마른 수건으로 액체를 닦아 낸 다음 완전 히 건조시킨 후 좌석 히터(열선)를 사용하십시오. • 좌석 히터(열선)는 많은 양의 전기 동력을 필요로 합 니다. 배터리가 방전되지 않도록 엔진 정지 상태에 서는 되도록 사용을 자제하십시오.
̰ 좌석 통풍
OCN7N033147 OCN7N033147 • 시동이 걸린 상태에서 해당 좌석 통풍 스위치를 작 동하면 해당 표시등이 켜지면서 해당 좌석에서 바 람이 나옵니다. • 좌석 통풍 조절 OFF → 약( ) → 중( ) → 강( ) ↑ • 좌석 통풍 작동 중 버튼을 1.5초 이상 누르면 좌석 통풍 을 끌 수 있습니다. (사양 적용 시) • 시동을 껐다가 다시 시동을 걸면 좌석 통풍은 항상 OFF 상태가 됩니다. 단, 운전석 통풍은 자동 쾌적 제어 기능이 켜져 있으 면 내/외기 온도 및 공조 설정 온도에 따라 ON/OFF 됩니다. • 운전석 자동 쾌적 제어 (사양 적용 시) 인포테인먼트 시스템 환경 설정 중 스

In [81]:
from langchain_core.documents import Document
docs = [Document(page_content = reulst[i], metadata = {"page_label": info[i]['page']}) for i in range(len(reulst))]

In [82]:
docs[0]

Document(metadata={'page_label': 1}, page_content='안전 및 차량 손상 경고\n본 취급설명서는 고객 및 차량의 안전과 관련한 심각한 위험과 제품 사용에 대한 올바른 정보를 사 전에 알리는 안전경고 표시입니다. 지시사항은 반드시 숙지하여 지켜주십시오. 경고, 주의표시 경고, 주의가 있는 문장 및 진하게 표시되어 있는 부분은 특히 유념하십시오. 주 의 차량이 고장나거나 손상될 우려가 있는 경우의 주의 표 시입니다. 경 고 사람이 다치거나 사망의 우려가 있는 경우의 경고 표시 입니다. i 알아두기 차량 용어 또는 추가 설명이 필요한 정보 표시입니다.\n선택 또는 미장착 사양표시 < >\n안전을 위해 반드시 지켜야 하는 금지 표시입니다.\n본 취급설명서에는 모든 트림모델 및 선택사양을 포함 하여 설명하고 있습니다. 따라서 고객님 차량에 장착되지 않은 사양이 설명될 수 있습니다.')

## 2. 스플릿

In [136]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)

chunks = splitter.split_documents(docs)
print(len(chunks), "전체 잘린 chunk 사이즈")

1520 전체 잘린 chunk 사이즈


In [137]:
chunks[100]

Document(metadata={'page_label': 34}, page_content='3-6\n̰ A타입 A타입\n̰ B타입 B타입\nOCN7N031130 OCN7N031130\nOCN7N031004 OCN7N031004\n주 의 • 전동식 좌석 조절 장치는 시동을 꺼도 작동합니다. 어 린이를 차 안에 혼자 둔 경우 예기치 못한 사고로 다칠 수 있습니다. • 조절장치는 모터로 작동됩니다. 조절이 끝나면 스위 치에서 손을 떼십시오. 과도한 조작은 모터의 전기 장치에 손상을 입힐 수 있습니다. • 전동식 좌석 조절장치는 많은 양의 전기 동력을 필 요로 합니다. 배터리가 방전되지 않도록 엔진 정지 상태에서는 되도록 사용을 자제하십시오.')

## 3. 벡터 스토어 저장

In [ ]:
db_path = "../vectorstore/chromadb_hyundai"
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

In [138]:
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory=db_path,
    collection_name="hyundai_2026"
)

print("벡터 저장소 저장 완료")

벡터 저장소 저장 완료


# 2. 검색

### 벡터 스토어 불러오기

In [100]:
# 벡터저장소가 이미 있는 상황

embedding = OpenAIEmbeddings(
    model="text-embedding-3-small"
)
persist_dir = "../vectorstore/chromadb_hyundai"
collection_name = "hyundai_2026"

vectorsore = Chroma(
    persist_directory=persist_dir,
    collection_name=collection_name,
    embedding_function=embedding
)

In [101]:
vectorsore._collection.count()

1520

## 검색기 만들기

In [116]:
ret_similarity = vectorsore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 10}
)

In [117]:
ret_mmr = vectorsore.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k" : 30}
)

In [118]:
from langchain_core.documents import Document

# 저장해둔 vectorsotre를  load_vectorsotre._collection 으로 가져온 뒤
# chunck와 동일한 형태인 Document 리스트로 만들기

db_docs = vectorsore._collection.get(include=["documents", "metadatas"])
bm_doc = []

for content, meta, id in zip(db_docs["documents"], db_docs["metadatas"], db_docs["ids"]):
    bm_doc.append(Document(page_content=content, metadata=meta, id=id))

In [119]:
from langchain_community.retrievers import BM25Retriever

bm25 = BM25Retriever.from_documents(bm_doc) # 키워드 기반 검색기
bm25.k = 10

In [120]:
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever

ret_hybrid = EnsembleRetriever(
    retrievers=[ret_similarity, bm25],
    weights=[0.7, 0.3]
)

In [121]:
question = "안전벨트 구속 장치"
result = ret_hybrid.invoke(question)
rich_docs(result, title='기업 비교')

                                                                                                                   
┏━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
┃      # ┃ Source         ┃       Page ┃ Preview                                                                   
┡━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
│      1 │                │         32 │ 안전 장치                                                                 
│      2 │                │         42 │ 안전벨트 ̰ 안전벨트 구속 장치 안전벨트 사용 중요성 3-14                    
│      3 │                │         46 │ 안전 장치 뒷좌석 중앙 OCN7030027 OCN7030027 플레이트(1)를 “찰칵” 소리가 날
│      4 │                │         29 │ 에어백 경고등............................................. 3-27 좌석 통풍.
│      5 │                │        432 │ 색인 선바이저 시계 카 매트 후크 컵 홀더 코트 후크 휴대폰 무선 충전 시스템 
│      6 │                │        432 │ 임산부의 안전벨트 착용 3-19 큰 어린이의 안전벨트 착용 3-19 안전벨트 착용 2
│      7 │                │         52 │ 안전 장치 에어백(보조 구속 장치) 1.      운전석 에어백 2.        동승석 에
│      8 │                │        432 │ 3-42 에어백 경고 라벨 주의 사항 3-42 차를 수리할 때, 폐차할 때 3-42 에어백
│      9 │                │         56 │ 에어백이 작동해서 탑승자를 보호하는 효과는 탑승자 가 안전벨트에 고정되어  
│     10 │                │         21 │ 안전벨트 착용 ̰ 탑승자를 보호하는 안전벨트 착용 02 2-3                     
│     11 │                │         29 │ 안전벨트 사용 시 유의 사항......................... 3-19 충돌 사고가 났을 
│     12 │                │         29 │ 안전벨트를 이용할 때.....................3-20 좌석 조절 (수동식)..........
└────────┴────────────────┴────────────┴───────────────────────────────────────────────────────────────────────────

# 3. LLM

In [180]:
# 1. 프롬프트
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """
     주어진 컨텍스트만 근거로 정확하게 대답해라
     컨텍스트에 없으면 문서에 "근거 없음" 이라고 말해라
     
     [출처 작성 규칙]
     - 대답에는 어떤 문서의 몇 페이지를 근거하고 있는지 각 문장별로 괄호 안에 작성해라
     - 여러개의 문서를 근거하고 있을 경우 누락이 모두 작성하라
     - 특정 문제 사항에 대한 질문일 경우 해결책을 단계별로 작성하라

     [컨텍스트]
     {context}
     """),
    ("user", "{question}"),
])

# 2. 모델 선택
model = ChatOpenAI(
    temperature=0,
    model = "gpt-4.1-mini",
    verbose=True
)

# 3. output parser 선택
outputparser = StrOutputParser()

# 4. chain 생성
chain = rag_prompt | model | outputparser


In [181]:
# 문서를 합치는 함수
def format_docs(docs):
    return "\n\---\n\n".join([f"컨텐츠: " + doc.page_content + f"\n\n페이지: {doc.metadata.get('page_label')}" for doc in docs])

In [182]:
rag_chain = (
    {
        "context" : ret_hybrid | RunnableLambda(format_docs),
        "question" : RunnablePassthrough()
    }
    | chain
)
rag_chain

{
  context: EnsembleRetriever(retrievers=[VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x00000249D359DE10>, search_kwargs={'k': 10}), BM25Retriever(vectorizer=<rank_bm25.BM25Okapi object at 0x00000249BE742590>, k=10)], weights=[0.7, 0.3])
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n     주어진 컨텍스트만 근거로 정확하게 대답해라\n     컨텍스트에 없으면 문서에 "근거 없음" 이라고 말해라\n\n     [출처 작성 규칙]\n     - 대답에는 어떤 문서의 몇 페이지를 근거하고 있는지 각 문장별로 괄호 안에 작성해라\n     - 여러개의 문서를 근거하고 있을 경우 누락이 모두 작성하라\n     - 특정 문제 사항에 대한 질문일 경우 해결책을 단계별로 작성하라\n\n     [컨텍스트]\n     {context}\n     '), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question

In [183]:
result = rag_chain.invoke("안전벨트 사용의 중요성은 무엇인가요?")
print(result)

안전벨트 사용의 중요성은 교통사고 발생 시 차량 탑승자의 생명과 신체를 보호하는 중요한 역할을 하기 때문입니다(페이지 29, 42, 43). 또한, 국내 교통법규에 따라 모든 탑승자는 안전벨트를 착용해야 하며, 안전벨트를 착용하지 않으면 경고등과 경고음이 작동하여 착용을 유도합니다(페이지 29, 432). 안전벨트는 사고 시 탑승자가 차량 내에서 안전하게 고정되어 부상 위험을 크게 줄여줍니다(페이지 42, 43). 따라서 안전벨트 착용은 생명 보호와 법적 의무 준수를 위해 매우 중요합니다(페이지 29, 42, 43).
